# PyTorch를 활용한 MPG 연비 예측 회귀 모델

이 노트북은 PyTorch를 사용하여 자동차의 MPG(Miles Per Gallon) 연비를 예측하는 회귀 모델을 구현합니다.

## 목표
- PyTorch의 신경망을 활용한 회귀 문제 해결
- 객체 지향 프로그래밍을 통한 모듈화된 코드 작성
- 데이터 전처리부터 모델 훈련까지의 전체 파이프라인 구축

## 1. 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (시각화를 위해)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

## 2. 신경망 모델 정의

MPG 예측을 위한 다층 퍼셉트론 모델을 정의합니다.

In [ ]:
class MPGRegressor(nn.Module):
    """연비 예측을 위한 신경망 모델"""
    
    def __init__(self, input_size=6):
        super(MPGRegressor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),  # 입력층
            nn.ReLU(),
            nn.Linear(64, 32),          # 은닉층 1
            nn.ReLU(),
            nn.Linear(32, 32),          # 은닉층 2
            nn.ReLU(),
            nn.Linear(32, 1)            # 출력층 (연비 예측값)
        )

    def forward(self, x):
        return self.layers(x)

# 모델 구조 확인
model = MPGRegressor(input_size=6)
print("모델 구조:")
print(model)

# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n총 파라미터 수: {total_params:,}")
print(f"훈련 가능한 파라미터 수: {trainable_params:,}")

## 3. 데이터 처리 클래스

MPG 데이터를 로드하고 전처리하는 클래스를 구현합니다.

In [ ]:
class MPGDataProcessor:
    """MPG 데이터 전처리 클래스"""
    
    def __init__(self, data_path):
        self.data_path = data_path
        self.scaler = StandardScaler()
        
    def load_and_preprocess_data(self):
        """데이터 로드 및 전처리"""
        # 데이터 로드
        mpg = pd.read_csv(self.data_path)
        print("원본 데이터 정보:")
        print(mpg.head())
        print(mpg.info())
        print(mpg.describe())
        
        # 결측값 제거
        mpg = mpg.dropna(how="any", axis=0)
        print(f"\n결측값 제거 후 데이터 크기: {mpg.shape}")
        
        # 특성과 타겟 분리
        X = mpg.iloc[:, 1:7]  # 특성 변수 (cylinders ~ origin)
        y = mpg.iloc[:, 0]    # 타겟 변수 (mpg)
        
        print(f"\n특성 데이터 형태: {X.shape}")
        print(f"타겟 데이터 형태: {y.shape}")
        print(f"\n특성 변수명: {list(X.columns)}")
        
        return X, y
    
    def create_data_loaders(self, X, y, test_size=0.2, batch_size=32, random_state=42):
        """데이터 로더 생성"""
        # 데이터 표준화
        X_scaled = self.scaler.fit_transform(X)
        
        # 훈련/테스트 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=test_size, random_state=random_state
        )
        
        print(f"훈련 데이터 크기: {X_train.shape}")
        print(f"테스트 데이터 크기: {X_test.shape}")
        
        # 텐서 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
        
        # 데이터셋 생성
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        
        # 데이터 로더 생성
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        return train_loader, test_loader, (X_train, X_test, y_train, y_test)

## 4. 데이터 로드 및 탐색적 데이터 분석

In [ ]:
# 데이터 처리 객체 생성 및 데이터 로드
data_processor = MPGDataProcessor("./data/mpg.csv")
X, y = data_processor.load_and_preprocess_data()

In [ ]:
# 데이터 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('MPG 데이터 특성별 분포', fontsize=16)

# 각 특성별 히스토그램
for i, column in enumerate(X.columns):
    row = i // 3
    col = i % 3
    axes[row, col].hist(X[column], bins=20, alpha=0.7, color='skyblue')
    axes[row, col].set_title(f'{column} 분포')
    axes[row, col].set_xlabel(column)
    axes[row, col].set_ylabel('빈도')

plt.tight_layout()
plt.show()

In [ ]:
# 타겟 변수(MPG) 분포 확인
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.hist(y, bins=20, alpha=0.7, color='lightcoral')
plt.title('MPG 분포')
plt.xlabel('MPG')
plt.ylabel('빈도')

plt.subplot(1, 2, 2)
plt.boxplot(y)
plt.title('MPG 박스플롯')
plt.ylabel('MPG')

plt.tight_layout()
plt.show()

print(f"MPG 통계:")
print(f"평균: {y.mean():.2f}")
print(f"표준편차: {y.std():.2f}")
print(f"최솟값: {y.min():.2f}")
print(f"최댓값: {y.max():.2f}")

In [ ]:
# 특성 간 상관관계 분석
plt.figure(figsize=(10, 8))
correlation_data = pd.concat([X, y], axis=1)
correlation_matrix = correlation_data.corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('특성 간 상관관계 히트맵')
plt.tight_layout()
plt.show()

# MPG와 각 특성 간의 상관계수 출력
print("MPG와 각 특성 간의 상관계수:")
mpg_correlations = correlation_matrix['mpg'].drop('mpg').sort_values(key=abs, ascending=False)
for feature, corr in mpg_correlations.items():
    print(f"{feature}: {corr:.3f}")

## 5. 데이터 로더 생성

In [ ]:
# 데이터 로더 생성
train_loader, test_loader, split_data = data_processor.create_data_loaders(X, y)
X_train, X_test, y_train, y_test = split_data

print(f"\n배치 크기: {train_loader.batch_size}")
print(f"훈련 배치 수: {len(train_loader)}")
print(f"테스트 배치 수: {len(test_loader)}")

## 6. 모델 훈련 클래스

모델 훈련과 평가를 담당하는 클래스를 정의합니다.

In [ ]:
class MPGTrainer:
    """모델 훈련 및 평가 클래스"""
    
    def __init__(self, model, learning_rate=0.001):
        self.model = model
        self.criterion = nn.MSELoss()  # 평균 제곱 오차
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        self.train_losses = []
        
    def train(self, train_loader, epochs=100):
        """모델 훈련"""
        print(f"모델 훈련 시작 (에포크: {epochs})")
        self.model.train()  # 훈련 모드 설정
        
        for epoch in range(epochs):
            total_loss = 0
            batch_count = 0
            
            for inputs, labels in train_loader:
                # 기울기 초기화
                self.optimizer.zero_grad()
                
                # 순전파
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                # 역전파 및 최적화
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
                batch_count += 1
            
            # 에포크별 평균 손실 저장
            avg_loss = total_loss / batch_count
            self.train_losses.append(avg_loss)
            
            if (epoch + 1) % 10 == 0:  # 10 에포크마다 출력
                print(f'에포크 {epoch+1}/{epochs}, 평균 손실: {avg_loss:.4f}')
        
        print("모델 훈련 완료!")
    
    def evaluate(self, test_loader):
        """모델 평가"""
        print("\n모델 평가 시작")
        self.model.eval()  # 평가 모드 설정
        
        with torch.no_grad():  # 기울기 계산 비활성화
            total_loss = 0
            total_samples = 0
            all_predictions = []
            all_labels = []
            
            for inputs, labels in test_loader:
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                total_samples += inputs.size(0)
                total_loss += loss.item() * inputs.size(0)
                
                all_predictions.extend(outputs.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            # MSE와 RMSE 계산
            avg_mse = total_loss / total_samples
            rmse = np.sqrt(avg_mse)
            
            print(f'테스트 데이터 평균 MSE: {avg_mse:.4f}')
            print(f'테스트 데이터 RMSE: {rmse:.4f}')
            
            return avg_mse, rmse, np.array(all_predictions), np.array(all_labels)
    
    def plot_training_history(self):
        """훈련 과정 시각화"""
        plt.figure(figsize=(10, 6))
        plt.plot(self.train_losses, label='Training Loss', color='blue')
        plt.title('훈련 과정의 손실 변화')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

## 7. 모델 훈련 실행

In [ ]:
# 새로운 모델 인스턴스 생성
model = MPGRegressor(input_size=X.shape[1])
print(f"모델 구조:")
print(model)

# 훈련 객체 생성
trainer = MPGTrainer(model, learning_rate=0.001)

# 모델 훈련
trainer.train(train_loader, epochs=100)

## 8. 훈련 과정 시각화

In [ ]:
# 훈련 과정 시각화
trainer.plot_training_history()

## 9. 모델 평가 및 결과 분석

In [ ]:
# 모델 평가
mse, rmse, predictions, true_values = trainer.evaluate(test_loader)

In [ ]:
# 예측 결과 시각화
plt.figure(figsize=(15, 5))

# 실제값 vs 예측값 산점도
plt.subplot(1, 3, 1)
plt.scatter(true_values, predictions, alpha=0.6, color='blue')
plt.plot([true_values.min(), true_values.max()], [true_values.min(), true_values.max()], 'r--', lw=2)
plt.xlabel('실제 MPG')
plt.ylabel('예측 MPG')
plt.title('실제값 vs 예측값')
plt.grid(True, alpha=0.3)

# 잔차 플롯
residuals = true_values.flatten() - predictions.flatten()
plt.subplot(1, 3, 2)
plt.scatter(predictions, residuals, alpha=0.6, color='green')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('예측값')
plt.ylabel('잔차 (실제값 - 예측값)')
plt.title('잔차 플롯')
plt.grid(True, alpha=0.3)

# 잔차 히스토그램
plt.subplot(1, 3, 3)
plt.hist(residuals, bins=20, alpha=0.7, color='orange')
plt.xlabel('잔차')
plt.ylabel('빈도')
plt.title('잔차 분포')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 통계적 분석
from scipy.stats import pearsonr
correlation, p_value = pearsonr(true_values.flatten(), predictions.flatten())

print(f"\n=== 모델 성능 분석 ===")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"상관계수: {correlation:.4f}")
print(f"결정계수 (R²): {correlation**2:.4f}")
print(f"평균 절대 오차 (MAE): {np.mean(np.abs(residuals)):.4f}")
print(f"잔차 평균: {np.mean(residuals):.4f}")
print(f"잔차 표준편차: {np.std(residuals):.4f}")

## 10. 예측 예시 및 모델 해석

In [ ]:
# 몇 가지 샘플 예측 결과 확인
model.eval()
with torch.no_grad():
    # 처음 10개 테스트 샘플 예측
    sample_inputs = torch.tensor(X_test[:10], dtype=torch.float32)
    sample_predictions = model(sample_inputs)
    
print("=== 샘플 예측 결과 ===")
print(f"{'인덱스':<5} {'실제값':<8} {'예측값':<8} {'오차':<8}")
print("-" * 35)

for i in range(10):
    actual = y_test.iloc[i]
    predicted = sample_predictions[i].item()
    error = abs(actual - predicted)
    print(f"{i:<5} {actual:<8.2f} {predicted:<8.2f} {error:<8.2f}")

In [ ]:
# 특성별 중요도 시각화 (간단한 분석)
feature_names = X.columns.tolist()
feature_importance = np.abs(correlation_matrix['mpg'].drop('mpg').values)

plt.figure(figsize=(10, 6))
indices = np.argsort(feature_importance)[::-1]
plt.bar(range(len(feature_importance)), feature_importance[indices], alpha=0.7)
plt.xticks(range(len(feature_importance)), [feature_names[i] for i in indices], rotation=45)
plt.title('특성별 MPG와의 상관관계 크기')
plt.ylabel('절대 상관계수')
plt.tight_layout()
plt.show()

print("특성 중요도 (절대 상관계수 기준):")
for i, idx in enumerate(indices):
    print(f"{i+1}. {feature_names[idx]}: {feature_importance[idx]:.3f}")

## 11. 결론 및 개선 방안

### 모델 성능 요약
- 본 모델은 PyTorch를 활용하여 자동차의 MPG를 예측하는 회귀 모델을 성공적으로 구현했습니다.
- 다층 퍼셉트론 구조를 사용하여 비선형 관계를 학습할 수 있었습니다.

### 개선 방안
1. **하이퍼파라미터 튜닝**: 학습률, 배치 크기, 네트워크 구조 최적화
2. **정규화 기법**: Dropout, Batch Normalization, L1/L2 정규화 적용
3. **교차 검증**: K-fold 교차 검증을 통한 더 안정적인 성능 평가
4. **앙상블 방법**: 여러 모델의 예측을 결합하여 성능 향상
5. **특성 엔지니어링**: 새로운 특성 생성 또는 특성 선택 기법 적용

In [ ]:
# 모델 저장 (선택사항)
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': trainer.optimizer.state_dict(),
    'train_losses': trainer.train_losses,
    'test_mse': mse,
    'test_rmse': rmse
}, 'mpg_regressor_model.pth')

print("모델이 'mpg_regressor_model.pth' 파일로 저장되었습니다.")